In [1]:
# IMPORT
import pandas as pd

In [ ]:
# DATA
data = pd.read_csv('train.csv')
print("Đã tải dữ liệu train.csv thành công!")
    
print("\n--- Tên tất cả các trường ---")
print(data.columns.tolist())

Đã tải dữ liệu train.csv thành công!

--- Tên tất cả các trường ---
['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF

TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
# 1. Nhóm Vị trí & Đặc điểm lô đất 

In [ ]:
# 2. Nhóm Quy mô & Không gian sử dụng 

In [ ]:
# 3. Nhóm Chất lượng & Tình trạng công trình 

In [ ]:
# 4. Nhóm Tiện ích, điều kiện bán & yếu tố thời gian 